# 04_Liquidity_Assessment.ipynb

## Title: "Liquidity Analysis"

### Contents:

- Liquidity metrics exploration from SEC data.
- Cross-checking with liquidity_query.sql.
- Computing liquidity ratios.
- Visualizing cash flow and liquidity trends.